In [76]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from functools import reduce
from collections import Counter
import random
from math import log
from sklearn.manifold import TSNE
import pandas as pd
import plotly.express as px

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

### Examples of preprocessing and tf-idf, PMI, etc.

In [3]:
def preproc(txt):
    #return txt.split()
    toks = txt.lower().split()
    toks = [t for t in toks if t not in stops]
    toks = [t for t in toks if t.isalpha()]
    return toks
    #ngms = []
    #for n in range(4):
    #    for ngm in ngrams(toks, n):
    #        ngms.append(' '.join(ngm))
    #return ngms

docs = [] # history, science
for l in open('argument-posts-sample.txt').readlines():
    post_num, topic, usr, post, meta = l.split('\t')
    sents = ' '.join(post.split(' ||| '))
    docs.append((topic, preproc(sents)))

In [13]:
science = reduce(lambda a,b: a+b,
                 [lst for topic, lst in docs
                  if topic == 'Science and Math'])
history = reduce(lambda a,b: a+b,
                 [lst for topic, lst in docs
                  if topic == 'History'])

In [8]:
print(random.sample(history, 100))
print(len(set(history)))

['was', 'fight', 'more', '...', 'this', '-LRB-', 'agent', 'difference', 'is', 'shouldas', 'never', 'country', 'thing', 'And', 'the', 'welfare', 'are', '...', 'in', 'the', 'conducting', '.', 'and', 'with', 'that', 'fact', 'were', 'stage', 'Joe', 'become', 'Off', 'Wilson', 'from', '17', 'I', ',', 'more', 'it', 'Asia', 'you', '880', 'hell', 'were', 'atheism', 'the', '?', ',', 'insane', 'say', 'all', 'have', 'about', 'is', 'this', 'international', 'him', 'The', 'administration', 'Jewish', 'an', 'for', 'and', 'the', 'book', ',', 'is', 'the', 'did', 'Again', 'expected', 'Strom', 'real', ',', 'the', 'a', '1802', 'for', 'around', 'it', 'nations', 'do', 'over', 'not', 'change', 'primary', 'between', 'if', 'country', 'holocaust', 'something', 'back', 'resisted', 'spending', 'less', 'persecution', 'to', 'Jews', '.', 'and', 'destructive']
8907


In [14]:
# find words which differentiate scientists from historians
# (but we aren't going to use a topic model
# cause pretend we don't know that yet)

print("Science Words")
for word, count in sorted(Counter(science).items(), 
                         key = lambda e:e[1], reverse=True)[:10]:
    print('%s\t%s'%(word, count))

print()

print("History Words")
for word, count in sorted(Counter(history).items(), 
                         key = lambda e:e[1], reverse=True)[:10]:
    print('%s\t%s'%(word, count))

Science Words
would	262
one	218
time	216
like	159
think	134
know	129
earth	125
universe	121
way	105
light	103

History Words
would	256
war	241
one	191
people	176
us	140
government	124
even	113
states	112
jews	110
american	107


In [18]:
def tf_idf(doc, corpus, min_count=100):
    tfs = Counter(doc)
    dfs = Counter(corpus)
    tfidfs = {w: tfs[w]/dfs[w] for w in
              [w for w in tfs if dfs[w] > min_count]}
    return tfidfs

print("Science Words")
for word, count in sorted(tf_idf(science, science+history).items(), 
                         key = lambda e:e[1], reverse=True)[:10]:
    print('%s\t%s'%(word, count))

print()

print("History Words")
for word, count in sorted(tf_idf(history, science+history).items(), 
                         key = lambda e:e[1], reverse=True)[:10]:
    print('%s\t%s'%(word, count))

print()

Science Words
universe	0.9918032786885246
light	0.9363636363636364
earth	0.9191176470588235
god	0.8118811881188119
life	0.8
wrong	0.7058823529411765
way	0.6774193548387096
time	0.6708074534161491
know	0.6683937823834197
like	0.6625

History Words
jews	1.0
war	0.9796747967479674
states	0.9572649572649573
government	0.9323308270676691
american	0.9304347826086956
us	0.7070707070707071
people	0.6901960784313725
world	0.6126126126126126
never	0.6078431372549019
first	0.5669291338582677



In [22]:
# find word collocations

def pmis(corpus, min_count=10):
    px = Counter(corpus) # probs of words on their own
    pxy = {} # probs of words occuring together
    for bg in ngrams(corpus, 2):
        #print(bg)
        if bg not in pxy:
            pxy[bg] = 0
        pxy[bg] += 1
    pmis = {}
    for bg in pxy:
        if pxy[bg] >= min_count:
            w1, w2 = bg
            pmi = log(pxy[bg]/px[w1])
            pmis[' '.join(bg)] = pmi
    return pmis

In [24]:
for word, count in sorted(pmis(science).items(), 
                         key = lambda e:e[1], reverse=True)[:50]:
    print('%s\t%s'%(word, count))

intellegent life	0.0
speed light	-0.6190392084062235
solar system	-0.6190392084062235
quran explanation	-0.6418538861723948
win prize	-0.6567795363890705
big bang	-0.6757554378480761
string theory	-0.7339691750802004
faster light	-0.7375989431307791
million years	-0.8690378470236094
behind door	-0.8823891801984737
prize behind	-0.9279867716373464
case choose	-1.3652409519220583
years ago	-1.5224265354444706
infinite number	-1.5533484457830569
make sense	-2.2300144001592104
would win	-3.1704492309627264


### Linguistic Features/Analysis with NLTK and Spacy

In [32]:
#sent = open('argument-posts-sample.txt').readlines()[5]
#sent = ' '.join(sent.split('\t')[3].split('|||'))
#sent = 'Take something into space though and it has no weight , but it still has mass , since it still has all its stuff .'
print(sent)
print()

# using NLTK
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn

print(sent_tokenize(sent))
for s in sent_tokenize(sent)[:1]:
    toks = word_tokenize(s)
    print(pos_tag(toks))
    sense = lesk(toks, 'space')
    print(sense)
    print(wn.synset('space.v.01').definition())

Take something into space though and it has no weight , but it still has mass , since it still has all its stuff .

['Take something into space though and it has no weight , but it still has mass , since it still has all its stuff .']
[('Take', 'VB'), ('something', 'NN'), ('into', 'IN'), ('space', 'NN'), ('though', 'IN'), ('and', 'CC'), ('it', 'PRP'), ('has', 'VBZ'), ('no', 'DT'), ('weight', 'NN'), (',', ','), ('but', 'CC'), ('it', 'PRP'), ('still', 'RB'), ('has', 'VBZ'), ('mass', 'NN'), (',', ','), ('since', 'IN'), ('it', 'PRP'), ('still', 'RB'), ('has', 'VBZ'), ('all', 'DT'), ('its', 'PRP$'), ('stuff', 'NN'), ('.', '.')]
Synset('space.v.01')
place at intervals


In [38]:
# Using spacy
import spacy

parser = spacy.load("en_core_web_sm")
parsed = parser(sent)
for tok in parsed:
    print('\t'.join([tok.text, tok.pos_, tok.lemma_]))

In [47]:
parsed = parser("I went to Washington today to get coffee with Denzel Washington")
for ent in parsed.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Washington 10 20 GPE
today 21 26 DATE
Denzel Washington 46 63 PERSON


### Example classifier using word embeddings

In [61]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np

embeddings = {}
for line in open('filtered_glove_embeddings.txt').readlines():
    w, v = line.strip().split(' ', 1)
    embeddings[w] = np.array([float(e) for e in v.split()])

In [69]:
def get_sparse_feature_matrix(words):
    _X = [{w: 1 for w in x} for x in words]
    dv = DictVectorizer()
    return dv.fit_transform(_X)

def get_embedding_feature_matrix(words, embed_dim = 300):
    X = np.zeros((len(words), embed_dim))
    for i, x in enumerate(words):
        for w in x:
            if w in embeddings:
                X[i, :] += embeddings[w]
    return X

data = [l.strip().split('\t') for l in open('wine.txt').readlines()]
W = [word_tokenize(review.lower()) for review, label in data]
y = [1 if len(label) > 3 else 0 for review, label in data]
#X = get_sparse_feature_matrix(W)
X = get_embedding_feature_matrix(W)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7021276595744681

### Plotting word embeddings

In [81]:
parser = spacy.load("en_core_web_sm")
vocab = []

for review, label in data:
    for tok in parser(review):
        if tok.pos_ == 'ADJ':
            vocab.append(tok.text)

M = []
names = []
for w in vocab:
    if w in embeddings:
        M.append(embeddings[w])
        names.append(w)

In [83]:
M = [embeddings[w] for w in vocab if w in embeddings]

In [78]:
#reducer = TSNE(n_components=2)
reducer = TruncatedSVD(n_components=2)
red = reducer.fit_transform(M)

In [79]:
dat = {'x': red[:,0], 'y': red[:,1], 'word': names}    
d = pd.DataFrame.from_dict(dat)
fig = px.scatter(d, x="x", y="y", hover_data=['word'])
fig.show()